In [57]:
import numpy as np
import pandas as pd

In [58]:
n_ag = 5
max_time_steps = 100
max_run_time = 8

In [59]:
c1 = np.random.randint(max_time_steps, size=max_time_steps*n_ag)
c2 = np.full(max_time_steps*n_ag, -1)
c3 = np.random.randint(n_ag, size=max_time_steps*n_ag)
c4 = np.full(max_time_steps*n_ag, False)
c5 = np.random.randint(1, max_run_time, size=max_time_steps*n_ag)
c6 = np.random.randint(max_run_time*2, max_time_steps, size=max_time_steps*n_ag)
c7 = [False] * max_time_steps * n_ag
df = pd.DataFrame({'time': c1, 'snd': c2, 'rcv': c3, 'sent': c4, 'run_time': c5, 'life_time': c6, 'done': c7}).sort_values(by=['time', 'rcv'])
# df = pd.DataFrame({'time': c1, 'snd': c2, 'rcv': c3, 'sent': c4, 'run_time': c5, 'life_time': c6}).sort_values(by=['time', 'rcv'])
df = df.drop_duplicates(subset=['time', 'snd', 'rcv'])
df

,time,snd,rcv,sent,run_time,life_time,done
414,0,-1,0,False,3,97,False
21,0,-1,1,False,7,90,False
150,0,-1,4,False,6,25,False
217,1,-1,0,False,6,89,False
20,1,-1,1,False,5,88,False
...,...,...,...,...,...,...,...
71,98,-1,4,False,1,49,False
22,99,-1,0,False,2,52,False
7,99,-1,1,False,6,36,False
311,99,-1,2,False,6,40,False


In [60]:
def gen_relations(n, min_route, max_route):
    b = np.random.randint(min_route, max_route, size=(n, n))
    res = (b + b.T) // 2
    np.fill_diagonal(res, 0)
    return res
relations = gen_relations(5, 0, 4)
print(relations)

[[0 2 1 1 2]
 [2 0 1 2 2]
 [1 1 0 1 1]
 [1 2 1 0 2]
 [2 2 1 2 0]]


In [61]:
df[df['snd'] < -1]

,time,snd,rcv,sent,run_time,life_time,done


In [62]:
def send_task(job, rcv):
    print(relations[rcv][job.rcv])
    tmp = [job.time + relations[rcv][job.rcv], job.rcv, rcv, False, job.run_time, job.life_time, False]
    print(job)
    print(tmp)
    df.loc[len(df)] = tmp
    df.drop(df.index[(df.time == job.time) & (df.snd == job.snd) & (df.rcv == job.rcv)], inplace = True)

In [63]:
send_task(df[(df.time == 0) & (df.snd == -1) & (df.rcv == 1)].squeeze(), 4)

2
time             0
snd             -1
rcv              1
sent         False
run_time         7
life_time       90
done         False
Name: 21, dtype: object
[2, 1, 4, False, 7, 90, False]


In [64]:
df[(df.time == 0) & (df.snd == -1) & (df.rcv == 1)]

,time,snd,rcv,sent,run_time,life_time,done


In [65]:
df[df['snd'] != -1]

,time,snd,rcv,sent,run_time,life_time,done
324,2,1,4,False,7,90,False


In [66]:
obs = {}
for i in range(5):
    obs[i]=[i-1,i,i+1]
obs

{0: [-1, 0, 1], 1: [0, 1, 2], 2: [1, 2, 3], 3: [2, 3, 4], 4: [3, 4, 5]}

In [67]:
[obs[x] for x in range(5) if relations[0][x] or x==0]

[[-1, 0, 1], [0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 5]]

In [68]:
df.loc[(df.time == 0) & (df.snd == 2) & (df.rcv == 4), 'done'] = True

In [69]:
df[(df.time == 0) & (df.snd == 2) & (df.rcv == 4)]

,time,snd,rcv,sent,run_time,life_time,done


In [77]:
df.loc[(df.time == 2) & (df.snd == 1) & (df.rcv == 4), 'done'] = True

In [78]:
df[(df.time == 2) & (df.snd == 1) & (df.rcv == 4)]

,time,snd,rcv,sent,run_time,life_time,done
324,2,1,4,False,7,90,True


In [83]:
df[df['done'] == True].shape[0]

1

In [134]:
num_jobs = [0]*5
total_run_time = [10,15,13,3,2]
result_num_jobs = (num_jobs-np.min(num_jobs))/np.ptp(num_jobs)
result_total_run_time = (total_run_time-np.min(total_run_time))/np.ptp(total_run_time)
print(result_num_jobs)
print(result_total_run_time)

[nan nan nan nan nan]
[0.61538462 1.         0.84615385 0.07692308 0.        ]


/var/folders/44/mb7l70216dl9vdpx1xfpzxp40000gn/T/ipykernel_57240/1459433522.py:3: RuntimeWarning: invalid value encountered in true_divide
  result_num_jobs = (num_jobs-np.min(num_jobs))/np.ptp(num_jobs)


In [92]:
out = [i for i in zip(result_num_jobs, result_total_run_time)]
out

[(0.0, 0.6153846153846154),
 (0.16666666666666666, 1.0),
 (0.5, 0.8461538461538461),
 (0.6666666666666666, 0.07692307692307693),
 (1.0, 0.0)]

In [126]:
np.sum([j for j in df[df.rcv == 2].run_time])

276

In [127]:
loc_obs = df[df.time <= 5]
loc_obs

,time,snd,rcv,sent,run_time,life_time,done
414,0,-1,0,False,3,97,False
150,0,-1,4,False,6,25,False
217,1,-1,0,False,6,89,False
20,1,-1,1,False,5,88,False
257,1,-1,2,False,6,73,False
54,1,-1,3,False,4,43,False
470,1,-1,4,False,6,36,False
472,2,-1,0,False,7,19,False
227,2,-1,1,False,2,74,False
128,2,-1,4,False,7,88,False


In [129]:
np.sum(loc_obs[loc_obs.rcv == 2].run_time)

8

In [125]:
loc_obs[loc_obs.rcv == 1].run_time.values[-1]

2

In [130]:
tmp = pd.Series({'time': 1,
                 'snd': 4,
                 'rcv': 2,
                 'sent': False,
                 'run_time': 3,
                 'life_time': 1,
                 'done': False})
df = pd.concat([df, tmp], ignore_index=True)
# self.jobs_df.loc[len(self.jobs_df)] = tmp
# print(job.time)
df.drop(df.index[(df.time == 2)
                 & (df.snd == 1)
                 & (df.rcv == 4)], inplace=True)

In [131]:
loc_obs[loc_obs.rcv == 2].life_time.values[-1]

48

In [133]:
np.sum([])

0.0

In [137]:
active_jobs_df = pd.DataFrame(columns=df.columns)
active_jobs_df

Index(['time', 'snd', 'rcv', 'sent', 'run_time', 'life_time', 'done', 0], dtype='object')

In [144]:
df[df.done == False]

,time,snd,rcv,sent,run_time,life_time,done,0
0,0.0,-1.0,0.0,False,3.0,97.0,False,NaN
1,0.0,-1.0,4.0,False,6.0,25.0,False,NaN
2,1.0,-1.0,0.0,False,6.0,89.0,False,NaN
3,1.0,-1.0,1.0,False,5.0,88.0,False,NaN
4,1.0,-1.0,2.0,False,6.0,73.0,False,NaN
...,...,...,...,...,...,...,...,...
318,98.0,-1.0,4.0,False,1.0,49.0,False,NaN
319,99.0,-1.0,0.0,False,2.0,52.0,False,NaN
320,99.0,-1.0,1.0,False,6.0,36.0,False,NaN
321,99.0,-1.0,2.0,False,6.0,40.0,False,NaN


In [154]:
df[df.rcv == 0].life_time.at[0]

97.0

In [293]:
print(df.head(10))
for i in range(5):
    col_index = df.columns.get_loc('life_time')
    # print(df.loc[df.rcv == i, 'life_time'].index[0], col_index)
    df.iloc[df.loc[df.rcv == i, 'life_time'].index[0], col_index] = df.loc[df.rcv == i, 'life_time'].values[0] - 1
print(df.head(10))

   time  snd  rcv   sent  run_time  life_time   done    0
0   0.0 -1.0  0.0  False       3.0       78.0  False  NaN
1   0.0 -1.0  4.0  False       6.0       77.0  False  NaN
2   1.0 -1.0  0.0  False       6.0       90.0  False  NaN
3   1.0 -1.0  1.0  False       5.0       78.0  False  NaN
4   1.0 -1.0  2.0  False       6.0       77.0  False  NaN
5   1.0 -1.0  3.0  False       4.0       77.0  False  NaN
6   1.0 -1.0  4.0  False       6.0       30.0  False  NaN
7   2.0 -1.0  0.0  False       7.0       90.0  False  NaN
8   2.0 -1.0  1.0  False       2.0       68.0  False  NaN
9   2.0 -1.0  4.0  False       7.0       82.0  False  NaN
   time  snd  rcv   sent  run_time  life_time   done    0
0   0.0 -1.0  0.0  False       3.0       77.0  False  NaN
1   0.0 -1.0  4.0  False       6.0       76.0  False  NaN
2   1.0 -1.0  0.0  False       6.0       90.0  False  NaN
3   1.0 -1.0  1.0  False       5.0       77.0  False  NaN
4   1.0 -1.0  2.0  False       6.0       76.0  False  NaN
5   1.0 -1.0  

In [291]:
df.loc[df.rcv == 1, 'life_time'].values[0]

79.0

In [287]:
df.index[0, 'life_time']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [251]:
df.loc[df.rcv == 0, 'life_time'].index[0], df.columns.get_loc('life_time')

(0, 5)

In [294]:
df.iloc[df.loc[df.rcv == 1, 'run_time'].index[0]]

time           1.0
snd           -1.0
rcv            1.0
sent         False
run_time       5.0
life_time     77.0
done         False
0              NaN
Name: 3, dtype: object

In [296]:
df.loc[df.rcv == 2, 'run_time'].index[0]

4

In [302]:
df.loc[df.rcv == 0, 'life_time'].index[1:]

Int64Index([  2,   7,  10,  14,  19,  23,  27,  29,  32,  37,  41,  48,  57,
             59,  63,  69,  77,  82,  86,  90,  93, 100, 110, 117, 120, 123,
            128, 133, 136, 139, 146, 149, 161, 165, 167, 173, 179, 185, 187,
            193, 197, 202, 212, 216, 220, 226, 231, 236, 240, 244, 246, 250,
            255, 260, 264, 270, 278, 284, 287, 295, 297, 304, 308, 312, 319],
           dtype='int64')

In [304]:
time_col_index = df.columns.get_loc('time')
time_col_index

0

In [310]:
print(df.head(10))
df.iloc[df.loc[df.rcv == 0, 'time'].index[1:], time_col_index] = \
    df.iloc[df.loc[df.rcv == 0, 'time'].index[1:], time_col_index] + 1
print(df.head(10))

   time  snd  rcv   sent  run_time  life_time   done    0
0   0.0 -1.0  0.0  False       3.0       77.0  False  NaN
1   0.0 -1.0  4.0  False       6.0       76.0  False  NaN
2   4.0 -1.0  0.0  False       6.0       90.0  False  NaN
3   1.0 -1.0  1.0  False       5.0       77.0  False  NaN
4   1.0 -1.0  2.0  False       6.0       76.0  False  NaN
5   1.0 -1.0  3.0  False       4.0       76.0  False  NaN
6   1.0 -1.0  4.0  False       6.0       30.0  False  NaN
7   5.0 -1.0  0.0  False       7.0       90.0  False  NaN
8   2.0 -1.0  1.0  False       2.0       68.0  False  NaN
9   2.0 -1.0  4.0  False       7.0       82.0  False  NaN
   time  snd  rcv   sent  run_time  life_time   done    0
0   0.0 -1.0  0.0  False       3.0       77.0  False  NaN
1   0.0 -1.0  4.0  False       6.0       76.0  False  NaN
2   5.0 -1.0  0.0  False       6.0       90.0  False  NaN
3   1.0 -1.0  1.0  False       5.0       77.0  False  NaN
4   1.0 -1.0  2.0  False       6.0       76.0  False  NaN
5   1.0 -1.0  